<h2>Coursera Capstone exercise - Part 2 </h2>

<h3>Summary:</h3>
<h3>Import csv file of location data of Mumbai neighborhoods that was created in Part 1 of the exercise</h3>
<h3>Explore Kabir's current neighborhood</h3>
<h3>For each shortlisted neighbourhood, get additional venue data of popular hangouts from Foursquare</h3>
<h3>Segment the city neighborhoods into various clusters, using the Kmeans algorithms</h3>
<h3>Conclusion</h3>
<h3></h3>

<h3>Importing and Preparing Mumbai Neighborhood Location Data</h3>

In [1]:
# import the required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
mum_hood_df = pd.read_csv('Mumbai_Hood_LL.csv')

In [3]:
mum_hood_df

,neighborhood,latitude,longitude,link
0,Zaveri Bazaar,18.951808,72.830697,https://en.wikipedia.org/wiki/Zaveri_Bazaar
1,Walkeshwar,18.947596,72.795957,https://en.wikipedia.org/wiki/Walkeshwar
2,Wadala,19.021632,72.864590,https://en.wikipedia.org/wiki/Wadala
3,Virar,19.470000,72.800000,https://en.wikipedia.org/wiki/Virar
4,Vidyavihar,19.080000,72.896000,https://en.wikipedia.org/wiki/Vidyavihar
5,"Versova, Mumbai",19.120000,72.820000,"https://en.wikipedia.org/wiki/Versova,_Mumbai"
6,Uran,18.890000,72.950000,https://en.wikipedia.org/wiki/Uran
7,Tardeo,18.969000,72.813000,https://en.wikipedia.org/wiki/Tardeo
8,Sunder Nagar,19.175000,72.842000,https://en.wikipedia.org/wiki/Sunder_Nagar
9,SPDC Colony,19.022350,72.554874,https://en.wikipedia.org/wiki/SPDC_Colony


In [4]:
mum_hood_df.shape

(97, 4)

In [5]:
mum_hood_df.drop([6], inplace=True)

In [6]:
mum_hood_df.reset_index(inplace=True, drop=True)

In [7]:
mum_hood_df.shape

(96, 4)

In [8]:
# getting the coordinates of A's current neighborhood - Prabhadevi
c_hood = 'Tardeo'
print(mum_hood_df.loc[mum_hood_df['neighborhood'] == c_hood, 'latitude'].values[0])
print(mum_hood_df.loc[mum_hood_df['neighborhood'] == c_hood, 'longitude'].values[0])

18.969
72.813


In [18]:
# converting column values to float
# df['MyColumnName'] = df['MyColumnName'].astype('float64') 


In [10]:
mum_hood_df

,neighborhood,latitude,longitude,link
0,Zaveri Bazaar,18.951808,72.830697,https://en.wikipedia.org/wiki/Zaveri_Bazaar
1,Walkeshwar,18.947596,72.795957,https://en.wikipedia.org/wiki/Walkeshwar
2,Wadala,19.021632,72.864590,https://en.wikipedia.org/wiki/Wadala
3,Virar,19.470000,72.800000,https://en.wikipedia.org/wiki/Virar
4,Vidyavihar,19.080000,72.896000,https://en.wikipedia.org/wiki/Vidyavihar
5,"Versova, Mumbai",19.120000,72.820000,"https://en.wikipedia.org/wiki/Versova,_Mumbai"
6,Tardeo,18.969000,72.813000,https://en.wikipedia.org/wiki/Tardeo
7,Sunder Nagar,19.175000,72.842000,https://en.wikipedia.org/wiki/Sunder_Nagar
8,SPDC Colony,19.022350,72.554874,https://en.wikipedia.org/wiki/SPDC_Colony
9,Shivaji Park Residential Zone,19.026724,72.838047,https://en.wikipedia.org/wiki/Shivaji_Park_Res...


<h3>Exploring Tardeo - Kabir's current neighborhood</h3>

In [11]:
# Define Foursquare credentials 
CLIENT_ID = 'VANBONR4EFDLXUOXYA3XQHRG4DN5TUJGF3U5ZUEVPMDP4J2L' # your Foursquare ID
CLIENT_SECRET = '43BEVVX03X3UUOIB2F1FG022EH5UXPUCZKOC01JMPINJJEWP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VANBONR4EFDLXUOXYA3XQHRG4DN5TUJGF3U5ZUEVPMDP4J2L
CLIENT_SECRET:43BEVVX03X3UUOIB2F1FG022EH5UXPUCZKOC01JMPINJJEWP


In [12]:
c_hood = 'Tardeo' # current neighborhood
neighborhood_latitude = mum_hood_df.loc[mum_hood_df['neighborhood'] == c_hood, 'latitude'].values[0] # neighborhood latitude value
neighborhood_longitude = mum_hood_df.loc[mum_hood_df['neighborhood'] == c_hood, 'longitude'].values[0] # neighborhood longitude value

neighborhood_name = c_hood # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Tardeo are 18.969, 72.813.


In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius 0.5 km

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VANBONR4EFDLXUOXYA3XQHRG4DN5TUJGF3U5ZUEVPMDP4J2L&client_secret=43BEVVX03X3UUOIB2F1FG022EH5UXPUCZKOC01JMPINJJEWP&v=20180605&ll=18.969,72.813&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c47f2b2351e3d7794d7f83d'},
 'response': {'headerLocation': 'Tardeo',
  'headerFullLocation': 'Tardeo, Mumbai',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 18.973500004500007,
    'lng': 72.81774952753321},
   'sw': {'lat': 18.964499995499995, 'lng': 72.8082504724668}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4edb8af964a52070f626e3',
       'name': 'Swati Snacks',
       'location': {'address': 'Karai Estate, Tardeo Road',
        'crossStreet': 'Opp Bhatia Hospital',
        'lat': 18.96644170253803,
        'lng': 72.81353124991814,
        'labeledLatLngs': [{'label': 'display',
          'lat': 18.96644170253803,
          'lng': 72.81353124991814}],
        'd

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Swati Snacks,Indian Restaurant,18.966442,72.813531
1,Theobroma,Bakery,18.970735,72.809816
2,Chinese Palace,Chinese Restaurant,18.972650,72.814187
3,Sardar Pav Bhaji,Snack Place,18.969740,72.815221
4,Oh! Calcutta,Bengali Restaurant,18.972468,72.815638


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


In [18]:
nearby_venues

,name,categories,lat,lng
0,Swati Snacks,Indian Restaurant,18.966442,72.813531
1,Theobroma,Bakery,18.970735,72.809816
2,Chinese Palace,Chinese Restaurant,18.972650,72.814187
3,Sardar Pav Bhaji,Snack Place,18.969740,72.815221
4,Oh! Calcutta,Bengali Restaurant,18.972468,72.815638
5,Maji Sagar,Fast Food Restaurant,18.972683,72.814683
6,Chung Fa,Chinese Restaurant,18.971102,72.814889
7,Under The Banyan Tree,Café,18.969328,72.808344
8,Cafe Coffee Day,Coffee Shop,18.966999,72.813983
9,Domino's Pizza,Pizza Place,18.969211,72.816193


<h3>A has shortlisted 5 neighborhood options he is considering moving to. Exploring if any of them matches Tardeo in terms of neighborhood composition</h3>

In [19]:
p_hoods = ('Tardeo', 'Shivaji Park', 'Powai', 'Lokhandwala Complex', 'Four Bungalows', 'Pali Hill') # potential neighborhoods

for p_hood in p_hoods:
    neighborhood_latitude = mum_hood_df.loc[mum_hood_df['neighborhood'] == p_hood, 'latitude'].values[0] # neighborhood latitude value
    neighborhood_longitude = mum_hood_df.loc[mum_hood_df['neighborhood'] == p_hood, 'longitude'].values[0] # neighborhood longitude value
    neighborhood_name = p_hood # neighborhood name
    print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))
    


Latitude and longitude values of Tardeo are 18.969, 72.813.
Latitude and longitude values of Shivaji Park are 19.026723999999998, 72.838047.
Latitude and longitude values of Powai are 19.12, 72.91.
Latitude and longitude values of Lokhandwala Complex are 19.130815, 72.82927.
Latitude and longitude values of Four Bungalows are 19.124714, 72.82721.
Latitude and longitude values of Pali Hill are 19.06587, 72.82674399999999.


In [20]:
mum_hood_shortlist = mum_hood_df[mum_hood_df['neighborhood'].isin(p_hoods)]

In [21]:
mum_hood_shortlist

,neighborhood,latitude,longitude,link
6,Tardeo,18.969000,72.813000,https://en.wikipedia.org/wiki/Tardeo
10,Shivaji Park,19.026724,72.838047,https://en.wikipedia.org/wiki/Shivaji_Park
16,Powai,19.120000,72.910000,https://en.wikipedia.org/wiki/Powai
20,Pali Hill,19.065870,72.826744,https://en.wikipedia.org/wiki/Pali_Hill
40,Lokhandwala Complex,19.130815,72.829270,https://en.wikipedia.org/wiki/Lokhandwala_Complex
54,Four Bungalows,19.124714,72.827210,https://en.wikipedia.org/wiki/Four_Bungalows


In [22]:
# explore select hoods

In [67]:
# Let's create a function to repeat the same process to all the neighborhoods in Mumbai
# radius set at 1 km for select neighborhoods, 500 m for all
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# Code to run the above function on each neighborhood and create a new dataframe called *mum_select_venues*.

LIMIT = 50
mum_select_venues = getNearbyVenues(names=mum_hood_shortlist['neighborhood'],
                                   latitudes=mum_hood_shortlist['latitude'],
                                   longitudes=mum_hood_shortlist['longitude']
                                  )


Tardeo
Shivaji Park
Powai
Pali Hill
Lokhandwala Complex
Four Bungalows


In [25]:
print(mum_select_venues.shape)
mum_select_venues.head()

(297, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tardeo,18.969,72.813,Swati Snacks,18.966442,72.813531,Indian Restaurant
1,Tardeo,18.969,72.813,Theobroma,18.970735,72.809816,Bakery
2,Tardeo,18.969,72.813,Chinese Palace,18.972650,72.814187,Chinese Restaurant
3,Tardeo,18.969,72.813,Oh! Calcutta,18.972468,72.815638,Bengali Restaurant
4,Tardeo,18.969,72.813,Willingdon Sports Club,18.976925,72.815256,Golf Course


In [26]:
## Let's check how many venues were returned for each neighborhood

In [27]:
mum_select_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Four Bungalows,47,47,47,47,47,47
Lokhandwala Complex,50,50,50,50,50,50
Pali Hill,50,50,50,50,50,50
Powai,50,50,50,50,50,50
Shivaji Park,50,50,50,50,50,50
Tardeo,50,50,50,50,50,50


In [28]:
print('There are {} uniques categories.'.format(len(mum_select_venues['Venue Category'].unique())))

There are 86 uniques categories.


In [406]:
### Analyse each neighborhood

In [29]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mum_select_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mum_select_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Bookstore,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,Goan Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Movie Theater,Multiplex,North Indian Restaurant,Other Great Outdoors,Park,Performing Arts Venue,Pizza Place,Playground,Plaza,Pub,Racetrack,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Spa,Stadium,Supermarket,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theater,Track,Vegetarian / Vegan Restaurant,Women's Store
0,Tardeo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Tardeo,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tardeo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Tardeo,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Tardeo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
mumbai_onehot.shape

(297, 87)

In [31]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,Accessories Store,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beach,Bed & Breakfast,Bengali Restaurant,Bookstore,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,Goan Restaurant,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Movie Theater,Multiplex,North Indian Restaurant,Other Great Outdoors,Park,Performing Arts Venue,Pizza Place,Playground,Plaza,Pub,Racetrack,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Spa,Stadium,Supermarket,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theater,Track,Vegetarian / Vegan Restaurant,Women's Store
0,Four Bungalows,0.021277,0.00,0.00,0.042553,0.00,0.00,0.00,0.085106,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.06383,0.042553,0.00,0.021277,0.06383,0.00,0.00,0.00,0.00,0.00,0.021277,0.00,0.021277,0.021277,0.00,0.00,0.00,0.021277,0.021277,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.021277,0.042553,0.00,0.00,0.042553,0.00,0.021277,0.00,0.021277,0.021277,0.042553,0.00,0.021277,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.042553,0.00,0.00,0.106383,0.00,0.021277,0.00,0.00,0.00,0.00,0.00,0.021277,0.00,0.021277,0.00,0.00,0.021277,0.00,0.00,0.00,0.00,0.00,0.06383,0.021277
1,Lokhandwala Complex,0.000000,0.02,0.00,0.020000,0.02,0.00,0.00,0.060000,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.06000,0.000000,0.00,0.000000,0.02000,0.02,0.00,0.00,0.00,0.00,0.000000,0.02,0.040000,0.020000,0.00,0.00,0.02,0.020000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.020000,0.00,0.00,0.080000,0.06,0.000000,0.02,0.040000,0.020000,0.080000,0.00,0.000000,0.02,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.060000,0.00,0.00,0.140000,0.00,0.020000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.02000,0.020000
2,Pali Hill,0.000000,0.02,0.04,0.020000,0.00,0.02,0.06,0.040000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.08000,0.020000,0.00,0.020000,0.00000,0.00,0.00,0.00,0.04,0.04,0.000000,0.06,0.000000,0.000000,0.02,0.02,0.04,0.000000,0.000000,0.02,0.02,0.00,0.02,0.00,0.00,0.00,0.000000,0.000000,0.00,0.02,0.100000,0.02,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.02,0.04,0.00,0.00,0.04,0.060000,0.00,0.000000,0.00,0.00,0.000000,0.02,0.00,0.00,0.00,0.00,0.00000,0.000000
3,Powai,0.000000,0.00,0.02,0.000000,0.00,0.00,0.04,0.040000,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.02000,0.040000,0.02,0.000000,0.02000,0.00,0.00,0.02,0.00,0.00,0.020000,0.02,0.000000,0.020000,0.00,0.00,0.04,0.000000,0.000000,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.020000,0.020000,0.02,0.04,0.140000,0.04,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.020000,0.00,0.02,0.020000,0.00,0.000000,0.06,0.00,0.00,0.02,0.02,0.020000,0.02,0.020000,0.02,0.00,0.000000,0.02,0.02,0.00,0.00,0.00,0.00000,0.000000
4,Shivaji Park,0.000000,0.00,0.02,0.000000,0.00,0.00,0.02,0.040000,0.04,0.00,0.00,0.00,0.04,0.00,0.02,0.04000,0.100000,0.00,0.000000,0.04000,0.00,0.00,0.00,0.00,0.02,0.000000,0.02,0.000000,0.000000,0.00,0.00,0.06,0.020000,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.000000,0.00,0.06,0.160000,0.02,0.000000,0.00,0.000000,0.000000,0.000000,0.02,0.000000,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.02,0.000000,0.02,0.02,0.000000,0.00,0.000000,0.02,0.00,0.00,0.00,0.00,0.000000,0.02,0.020000,0.00,0.00,0.000000,0.00,0.00,0.02,0.00,0.02,0.04000,0.000000
5,Tardeo,0.000000,0.00

In [33]:
mumbai_grouped.shape

(6, 87)

In [391]:
#### Let's print each neighborhood along with the top 10 most common venues

In [34]:
num_top_venues = 10

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Four Bungalows----
                           venue  freq
0                            Pub  0.11
1                            Bar  0.09
2                           Café  0.06
3  Vegetarian / Vegan Restaurant  0.06
4                    Coffee Shop  0.06
5                         Lounge  0.04
6               Asian Restaurant  0.04
7           Gym / Fitness Center  0.04
8                    Pizza Place  0.04
9              Indian Restaurant  0.04


----Lokhandwala Complex----
                venue  freq
0                 Pub  0.14
1              Lounge  0.08
2   Indian Restaurant  0.08
3  Italian Restaurant  0.06
4                 Bar  0.06
5                Café  0.06
6         Pizza Place  0.06
7               Diner  0.04
8           Juice Bar  0.04
9           Multiplex  0.04


----Pali Hill----
                  venue  freq
0     Indian Restaurant  0.10
1                  Café  0.08
2          Dessert Shop  0.06
3    Seafood Restaurant  0.06
4                Bakery  0.06
5         

In [35]:
# Let's put that into a *pandas* dataframe

In [77]:
# sort
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
# create dataframe to display top 15 venues per neighborhood
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Four Bungalows,Pub,Bar,Vegetarian / Vegan Restaurant,Café,Coffee Shop,Gym / Fitness Center,Asian Restaurant,Chinese Restaurant,Lounge,Pizza Place,Indian Restaurant,Liquor Store,Juice Bar,Japanese Restaurant,Fish Market,Gym,Food Truck,Donut Shop,Diner,Department Store
1,Lokhandwala Complex,Pub,Lounge,Indian Restaurant,Pizza Place,Bar,Café,Italian Restaurant,Juice Bar,Diner,Multiplex,Women's Store,Vegetarian / Vegan Restaurant,Liquor Store,Jewelry Store,Donut Shop,Fish Market,Fast Food Restaurant,Dessert Shop,Comfort Food Restaurant,Coffee Shop
2,Pali Hill,Indian Restaurant,Café,Dessert Shop,Bakery,Seafood Restaurant,Salad Place,Fast Food Restaurant,Scenic Lookout,Bar,Cupcake Shop,Arcade,Deli / Bodega,Pub,Park,Cocktail Bar,Italian Restaurant,Event Space,Chinese Restaurant,Burger Joint,General Entertainment
3,Powai,Indian Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Chinese Restaurant,Bar,Bakery,Fast Food Restaurant,Hotel,Gym,Grocery Store,Pizza Place,Creperie,Coffee Shop,Lounge,Department Store,Dessert Shop,Gastropub,Clothing Store,Mediterranean Restaurant
4,Shivaji Park,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Ice Cream Shop,Movie Theater,Coffee Shop,Breakfast Spot,Café,Bar,Beach,Vegetarian / Vegan Restaurant,Italian Restaurant,Dessert Shop,Deli / Bodega,Playground,Plaza,Burger Joint,Performing Arts Venue,Restaurant,Shopping Mall


In [38]:
### CLUSTER HOODS

In [39]:
# set number of clusters
kclusters = 3

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 0, 2, 1, 1, 1], dtype=int32)

In [40]:
## new df that includes cluster and top venues data

mumbai_merged = mum_hood_shortlist
# add clustering labels
mumbai_merged['Cluster Labels'] = kmeans.labels_
# merge mumbai_grouped with mumbai_merged_data to add latitude/longitude for each neighborhood
mumbai_cons = mumbai_merged.merge(neighborhoods_venues_sorted, left_on='neighborhood', right_on='Neighborhood', how='outer')

mumbai_cons # check the last columns!


/Users/shubhadeepbhattacharya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,neighborhood,latitude,longitude,link,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Tardeo,18.969000,72.813000,https://en.wikipedia.org/wiki/Tardeo,0,Tardeo,Bar,Bakery,Pizza Place,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Coffee Shop,Indian Restaurant,Ice Cream Shop,Restaurant,Lounge,Japanese Restaurant,Italian Restaurant,Hotel,Golf Course,Women's Store,Donut Shop,Movie Theater,Department Store,Deli / Bodega
1,Shivaji Park,19.026724,72.838047,https://en.wikipedia.org/wiki/Shivaji_Park,0,Shivaji Park,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Ice Cream Shop,Movie Theater,Coffee Shop,Breakfast Spot,Café,Bar,Beach,Vegetarian / Vegan Restaurant,Italian Restaurant,Dessert Shop,Deli / Bodega,Playground,Plaza,Burger Joint,Performing Arts Venue,Restaurant,Shopping Mall
2,Powai,19.120000,72.910000,https://en.wikipedia.org/wiki/Powai,2,Powai,Indian Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Chinese Restaurant,Bar,Bakery,Fast Food Restaurant,Hotel,Gym,Grocery Store,Pizza Place,Creperie,Coffee Shop,Lounge,Department Store,Dessert Shop,Gastropub,Clothing Store,Mediterranean Restaurant
3,Pali Hill,19.065870,72.826744,https://en.wikipedia.org/wiki/Pali_Hill,1,Pali Hill,Indian Restaurant,Café,Dessert Shop,Bakery,Seafood Restaurant,Salad Place,Fast Food Restaurant,Scenic Lookout,Bar,Cupcake Shop,Arcade,Deli / Bodega,Pub,Park,Cocktail Bar,Italian Restaurant,Event Space,Chinese Restaurant,Burger Joint,General Entertainment
4,Lokhandwala Complex,19.130815,72.829270,https://en.wikipedia.org/wiki/Lokhandwala_Complex,1,Lokhandwala Complex,Pub,Lounge,Indian Restaurant,Pizza Place,Bar,Café,Italian Restaurant,Juice Bar,Diner,Multiplex,Women's Store,Vegetarian / Vegan Restaurant,Liquor Store,Jewelry Store,Donut Shop,Fish Market,Fast Food Restaurant,Dessert Shop,Comfort Food Restaurant,Coffee Shop
5,Four Bungalows,19.124714,72.827210,https://en.wikipedia.org/wiki/Four_Bungalows,1,Four Bungalows,Pub,Bar,Vegetarian / Vegan Restaurant,Café,Coffee Shop,Gym / Fitness Center,Asian Restaurant,Chinese Restaurant,Lounge,Pizza Place,Indian Restaurant,Liquor Store,Juice Bar,Japanese Restaurant,Fish Market,Gym,Food Truck,Donut Shop,Diner,Department Store


In [55]:
mumbai_cons[['neighborhood','Cluster Labels']]


,neighborhood,Cluster Labels
0,Tardeo,0
1,Shivaji Park,0
2,Powai,2
3,Pali Hill,1
4,Lokhandwala Complex,1
5,Four Bungalows,1


In [56]:
# create map
map_clusters = folium.Map(location=[19.0760, 72.8777], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_cons['latitude'], mumbai_cons['longitude'], mumbai_cons['neighborhood'], mumbai_cons['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
### examine clusters

In [58]:
mumbai_cons.head()

,neighborhood,latitude,longitude,link,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Tardeo,18.969000,72.813000,https://en.wikipedia.org/wiki/Tardeo,0,Tardeo,Bar,Bakery,Pizza Place,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Coffee Shop,Indian Restaurant,Ice Cream Shop,Restaurant,Lounge,Japanese Restaurant,Italian Restaurant,Hotel,Golf Course,Women's Store,Donut Shop,Movie Theater,Department Store,Deli / Bodega
1,Shivaji Park,19.026724,72.838047,https://en.wikipedia.org/wiki/Shivaji_Park,0,Shivaji Park,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Ice Cream Shop,Movie Theater,Coffee Shop,Breakfast Spot,Café,Bar,Beach,Vegetarian / Vegan Restaurant,Italian Restaurant,Dessert Shop,Deli / Bodega,Playground,Plaza,Burger Joint,Performing Arts Venue,Restaurant,Shopping Mall
2,Powai,19.120000,72.910000,https://en.wikipedia.org/wiki/Powai,2,Powai,Indian Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Chinese Restaurant,Bar,Bakery,Fast Food Restaurant,Hotel,Gym,Grocery Store,Pizza Place,Creperie,Coffee Shop,Lounge,Department Store,Dessert Shop,Gastropub,Clothing Store,Mediterranean Restaurant
3,Pali Hill,19.065870,72.826744,https://en.wikipedia.org/wiki/Pali_Hill,1,Pali Hill,Indian Restaurant,Café,Dessert Shop,Bakery,Seafood Restaurant,Salad Place,Fast Food Restaurant,Scenic Lookout,Bar,Cupcake Shop,Arcade,Deli / Bodega,Pub,Park,Cocktail Bar,Italian Restaurant,Event Space,Chinese Restaurant,Burger Joint,General Entertainment
4,Lokhandwala Complex,19.130815,72.829270,https://en.wikipedia.org/wiki/Lokhandwala_Complex,1,Lokhandwala Complex,Pub,Lounge,Indian Restaurant,Pizza Place,Bar,Café,Italian Restaurant,Juice Bar,Diner,Multiplex,Women's Store,Vegetarian / Vegan Restaurant,Liquor Store,Jewelry Store,Donut Shop,Fish Market,Fast Food Restaurant,Dessert Shop,Comfort Food Restaurant,Coffee Shop


In [59]:
# cluster 1

In [60]:
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 0, mumbai_cons.columns[list(range(5, mumbai_cons.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Tardeo,Bar,Bakery,Pizza Place,Chinese Restaurant,Dessert Shop,Fast Food Restaurant,Coffee Shop,Indian Restaurant,Ice Cream Shop,Restaurant,Lounge,Japanese Restaurant,Italian Restaurant,Hotel,Golf Course,Women's Store,Donut Shop,Movie Theater,Department Store,Deli / Bodega
1,Shivaji Park,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Ice Cream Shop,Movie Theater,Coffee Shop,Breakfast Spot,Café,Bar,Beach,Vegetarian / Vegan Restaurant,Italian Restaurant,Dessert Shop,Deli / Bodega,Playground,Plaza,Burger Joint,Performing Arts Venue,Restaurant,Shopping Mall


In [61]:
# cluster 2

In [62]:
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 1, mumbai_cons.columns[list(range(5, mumbai_cons.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
3,Pali Hill,Indian Restaurant,Café,Dessert Shop,Bakery,Seafood Restaurant,Salad Place,Fast Food Restaurant,Scenic Lookout,Bar,Cupcake Shop,Arcade,Deli / Bodega,Pub,Park,Cocktail Bar,Italian Restaurant,Event Space,Chinese Restaurant,Burger Joint,General Entertainment
4,Lokhandwala Complex,Pub,Lounge,Indian Restaurant,Pizza Place,Bar,Café,Italian Restaurant,Juice Bar,Diner,Multiplex,Women's Store,Vegetarian / Vegan Restaurant,Liquor Store,Jewelry Store,Donut Shop,Fish Market,Fast Food Restaurant,Dessert Shop,Comfort Food Restaurant,Coffee Shop
5,Four Bungalows,Pub,Bar,Vegetarian / Vegan Restaurant,Café,Coffee Shop,Gym / Fitness Center,Asian Restaurant,Chinese Restaurant,Lounge,Pizza Place,Indian Restaurant,Liquor Store,Juice Bar,Japanese Restaurant,Fish Market,Gym,Food Truck,Donut Shop,Diner,Department Store


In [63]:
# cluster 3

In [64]:
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 2, mumbai_cons.columns[list(range(5, mumbai_cons.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,Powai,Indian Restaurant,Restaurant,Italian Restaurant,Ice Cream Shop,Chinese Restaurant,Bar,Bakery,Fast Food Restaurant,Hotel,Gym,Grocery Store,Pizza Place,Creperie,Coffee Shop,Lounge,Department Store,Dessert Shop,Gastropub,Clothing Store,Mediterranean Restaurant


<h3>Segment all Mumbai neighborhoods into various clusters, using the Kmeans algorithms</h3>

In [65]:
mum_hood_df.head()

,neighborhood,latitude,longitude,link
0,Zaveri Bazaar,18.951808,72.830697,https://en.wikipedia.org/wiki/Zaveri_Bazaar
1,Walkeshwar,18.947596,72.795957,https://en.wikipedia.org/wiki/Walkeshwar
2,Wadala,19.021632,72.864590,https://en.wikipedia.org/wiki/Wadala
3,Virar,19.470000,72.800000,https://en.wikipedia.org/wiki/Virar
4,Vidyavihar,19.080000,72.896000,https://en.wikipedia.org/wiki/Vidyavihar


In [66]:
mum_hood_df.shape

(96, 4)

In [68]:
LIMIT = 50
mum_select_venues = getNearbyVenues(names=mum_hood_df['neighborhood'],
                                   latitudes=mum_hood_df['latitude'],
                                   longitudes=mum_hood_df['longitude']
                                  )


Zaveri Bazaar
Walkeshwar
Wadala
Virar
Vidyavihar
Versova, Mumbai
Tardeo
Sunder Nagar
SPDC Colony
Shivaji Park Residential Zone
Shivaji Park
Shastri Nagar, Goregaon
Sewri
SEEPZ
Pydhonie
Prabhadevi
Powai
Poisar
Pedder Road
Parel
Pali Hill
Royal Opera House (Mumbai)
Navy Nagar
Nariman Point
Nala Sopara
Mira Road
Mazagaon
Matunga Road, Mumbai
Marol
Marine Lines
Marine Drive, Mumbai
Manori
Mandvi, Mumbai
Malad
Malabar Hill
Mahul
Mahim
Maheshwari Udyan, Mumbai
Mahalaxmi, Mumbai
Madh Island
Lokhandwala Complex
Lallubhai Compound
Kherwadi
Khar Danda
Kamathipura
Kalbadevi
Irla
I.C. Colony
Hiranandani Gardens, Mumbai
Hindu colony
Guru Tegh Bahadur Nagar
Gowalia Tank
Gorai
Girgaon
Four Bungalows
Fort (Mumbai precinct)
Ferry Wharf
Fanas Wadi
Dongri
Dindoshi
Dhobitalao
Dharavi
Deonar
Dava Bazaar
Dagdi Chawl
Dadar Parsi Colony
Dadar
D.N. Nagar
Currey Road railway station
Cumbala Hill
Cuffe Parade
Cotton Green
Colaba
Churchgate
Chor Bazaar
Chira Bazaar
Chinchpokli
Chandanwadi, Mumbai
Cavel
C.G.S. col

In [69]:
print(mum_select_venues.shape)
mum_select_venues.head()

(1410, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zaveri Bazaar,18.951808,72.830697,Bhagat Tarachand Restaurant,18.951802,72.830486,Indian Restaurant
1,Zaveri Bazaar,18.951808,72.830697,Shree Thaker Bhojnalay,18.951217,72.828326,Indian Restaurant
2,Zaveri Bazaar,18.951808,72.830697,Surti Hotel,18.952567,72.829905,Food
3,Zaveri Bazaar,18.951808,72.830697,Mohd Ali Road,18.954751,72.833198,BBQ Joint
4,Zaveri Bazaar,18.951808,72.830697,Suleman Mithaiwala,18.954728,72.833186,Dessert Shop


In [70]:
mum_select_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agripada,6,6,6,6,6,6
Altamount Road,30,30,30,30,30,30
"Amboli, Mumbai",5,5,5,5,5,5
Amrut Nagar,43,43,43,43,43,43
Antop Hill,4,4,4,4,4,4
Anushakti Nagar,5,5,5,5,5,5
Asalfa,3,3,3,3,3,3
Badhwar Park,34,34,34,34,34,34
Baiganwadi,1,1,1,1,1,1


In [71]:
print('There are {} uniques categories.'.format(len(mum_select_venues['Venue Category'].unique())))

There are 168 uniques categories.


In [72]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mum_select_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mum_select_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Café,Campground,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Lake,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,New American Restaurant,Nightclub,North Indian Restaurant,Opera House,Other Great Outdoors,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Tourist Information Center,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Zaveri Bazaar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Zaveri Bazaar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Zaveri Bazaar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Zaveri Bazaar,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Zaveri Bazaar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [73]:
mumbai_onehot.shape

(1410, 169)

In [74]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Café,Campground,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Lake,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,New American Restaurant,Nightclub,North Indian Restaurant,Opera House,Other Great Outdoors,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Pool,Pub,Racetrack,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Tourist Information Center,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Agripada,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
1,Altamount Road,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.066667,0.000000,0.000000,0.033333,0.033333,0.000,0.000000,0.000000,0.033333,0.1

In [75]:
mumbai_grouped.shape

(92, 169)

In [76]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agripada----
               venue  freq
0             Bakery  0.33
1  Indian Restaurant  0.17
2         Restaurant  0.17
3        Coffee Shop  0.17
4                Gym  0.17


----Altamount Road----
         venue  freq
0         Café  0.13
1       Bakery  0.07
2  Pizza Place  0.07
3    Bookstore  0.07
4   Restaurant  0.03


----Amboli, Mumbai----
                  venue  freq
0    Chinese Restaurant   0.4
1     Indian Restaurant   0.2
2           Coffee Shop   0.2
3        Sandwich Place   0.2
4  Other Great Outdoors   0.0


----Amrut Nagar----
                  venue  freq
0     Indian Restaurant  0.14
1                  Café  0.09
2            Restaurant  0.07
3  Fast Food Restaurant  0.05
4      Asian Restaurant  0.05


----Antop Hill----
                  venue  freq
0     Indian Restaurant  0.25
1                 Trail  0.25
2  Gym / Fitness Center  0.25
3         Grocery Store  0.25
4           Opera House  0.00


----Anushakti Nagar----
               venue  freq
0  Indian

                        venue  freq
0                      Resort  0.33
1           Indian Restaurant  0.17
2          Seafood Restaurant  0.17
3                    Aquarium  0.17
4  Tourist Information Center  0.17


----Gowalia Tank----
        venue  freq
0        Café  0.11
1  Hobby Shop  0.05
2         Spa  0.05
3     Brewery  0.05
4   Bookstore  0.05


----Guru Tegh Bahadur Nagar----
               venue  freq
0  Indian Restaurant  0.50
1      Train Station  0.17
2              Train  0.17
3                Gym  0.17
4           Pharmacy  0.00


----Hindu colony----
               venue  freq
0        Coffee Shop  0.12
1  Indian Restaurant  0.06
2             Lounge  0.06
3               Park  0.06
4     Farmers Market  0.06


----Hiranandani Gardens, Mumbai----
               venue  freq
0  Indian Restaurant  0.14
1        Pizza Place  0.06
2         Restaurant  0.06
3                Bar  0.06
4        Coffee Shop  0.04


----I.C. Colony----
                venue  freq
0         

                venue  freq
0  Chinese Restaurant  0.15
1         Coffee Shop  0.08
2   Indian Restaurant  0.08
3      Ice Cream Shop  0.08
4         Pizza Place  0.08


----Versova, Mumbai----
                venue  freq
0        Cocktail Bar   0.5
1                 Bar   0.5
2                 ATM   0.0
3         Opera House   0.0
4  Mughlai Restaurant   0.0


----Vidyavihar----
                  venue  freq
0  Fast Food Restaurant   0.4
1           Coffee Shop   0.2
2         Train Station   0.2
3                   Bar   0.2
4                   ATM   0.0


----Virar----
                            venue  freq
0                          Bakery   1.0
1                             ATM   0.0
2                     Opera House   0.0
3              Mughlai Restaurant   0.0
4  Multicuisine Indian Restaurant   0.0


----Wadala----
                  venue  freq
0     Indian Restaurant  0.25
1                 Trail  0.25
2  Gym / Fitness Center  0.25
3         Grocery Store  0.25
4           Op

In [78]:
# create dataframe to display top 15 venues per neighborhood
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agripada,Bakery,Indian Restaurant,Coffee Shop,Gym,Restaurant,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dhaba
1,Altamount Road,Café,Pizza Place,Bakery,Bookstore,Japanese Restaurant,Furniture / Home Store,Snack Place,Breakfast Spot,Brewery,Sandwich Place,Salon / Barbershop,Bus Station,Restaurant,Indian Restaurant,Italian Restaurant
2,"Amboli, Mumbai",Chinese Restaurant,Indian Restaurant,Coffee Shop,Sandwich Place,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Food
3,Amrut Nagar,Indian Restaurant,Café,Restaurant,Asian Restaurant,Shopping Mall,Electronics Store,Fast Food Restaurant,Bowling Alley,Brewery,Sandwich Place,Mediterranean Restaurant,Clothing Store,Bookstore,Falafel Restaurant,Chinese Restaurant
4,Antop Hill,Indian Restaurant,Gym / Fitness Center,Trail,Grocery Store,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dessert Shop,Diner


In [79]:
### CLUSTER HOODS

In [81]:
# set number of clusters
kclusters = 6

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([2, 2, 5, 2, 5, 5, 5, 2, 0, 2, 2, 5, 5, 2, 2, 2, 5, 2, 2, 5, 5, 2,
       3, 2, 3, 2, 5, 2, 5, 2, 5, 5, 2, 2, 5, 2, 2, 5, 0, 5, 2, 2, 2, 2,
       5, 2, 2, 2, 2, 5, 5, 5, 2, 2, 2, 1, 5, 5, 2, 2, 5, 0, 2, 2, 2, 5,
       2, 2, 0, 2, 4, 2, 5, 2, 2, 2, 2, 5, 2, 2, 5, 2, 5, 5, 5, 2, 2, 2,
       4, 5, 2, 5], dtype=int32)

In [85]:
## new df that includes cluster and top venues data

mumbai_merged = mum_hood_df
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_
# merge mumbai_grouped with mumbai_merged_data to add latitude/longitude for each neighborhood
mumbai_cons = neighborhoods_venues_sorted.merge(mumbai_merged, left_on='Neighborhood', right_on='neighborhood', how='outer')

mumbai_cons # check the last columns!

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
0,Agripada,Bakery,Indian Restaurant,Coffee Shop,Gym,Restaurant,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dhaba,2.0,Agripada,18.977700,72.827300,https://en.wikipedia.org/wiki/Agripada
1,Altamount Road,Café,Pizza Place,Bakery,Bookstore,Japanese Restaurant,Furniture / Home Store,Snack Place,Breakfast Spot,Brewery,Sandwich Place,Salon / Barbershop,Bus Station,Restaurant,Indian Restaurant,Italian Restaurant,2.0,Altamount Road,18.965200,72.808700,https://en.wikipedia.org/wiki/Altamount_Road
2,"Amboli, Mumbai",Chinese Restaurant,Indian Restaurant,Coffee Shop,Sandwich Place,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Food,5.0,"Amboli, Mumbai",19.129300,72.843400,"https://en.wikipedia.org/wiki/Amboli,_Mumbai"
3,Amrut Nagar,Indian Restaurant,Café,Restaurant,Asian Restaurant,Shopping Mall,Electronics Store,Fast Food Restaurant,Bowling Alley,Brewery,Sandwich Place,Mediterranean Restaurant,Clothing Store,Bookstore,Falafel Restaurant,Chinese Restaurant,2.0,Amrut Nagar,19.102077,72.912835,https://en.wikipedia.org/wiki/Amrut_Nagar
4,Antop Hill,Indian Restaurant,Gym / Fitness Center,Trail,Grocery Store,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dessert Shop,Diner,5.0,Antop Hill,19.022220,72.866564,https://en.wikipedia.org/wiki/Antop_Hill
5,Anushakti Nagar,Indian Restaurant,Convenience Store,Lake,Playground,Creperie,Cupcake Shop,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Cosmetics Shop,5.0,Anushakti Nagar,19.036390,72.922500,https://en.wikipedia.org/wiki/Anushakti_Nagar
6,Asalfa,Campground,Motorcycle Shop,Indian Restaurant,Electronics Store,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Zoo,Food Court,Donut Shop,Diner,5.0,Asalfa,19.091000,72.901000,https://en.wikipedia.org/wiki/Asalfa
7,Badhwar Park,Indian Restaurant,Café,Coffee Shop,Diner,Tennis Court,Restaurant,Hotel,German Restaurant,Falafel Restaurant,Middle Eastern Restaurant,Flea Market,Furniture / Home Store,Chaat Place,Chinese Restaurant,Stadium,2.0,Badhwar Park,18.921144,72.827325,https://en.wikipedia.org/wiki/Badhwar_Park
8,Baiganwadi,ATM,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Yoga Studio,Diner,Dhaba,Dessert Shop,0.0,Baiganwadi,19.061300,72.925800,https://en.wikipedia.org/wiki/Baiganwadi
9,Ballard Estate,Convenience Store,Harbor / Marina,Hotel,Home Service,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Food,Dhaba,2.0,Ballard Estate,18.950000,72.840000,https://en.wikipedia.org/wiki/Ballard_Estate


In [93]:
mumbai_cons[['Neighborhood', 'Cluster Labels' ]]

,Neighborhood,Cluster Labels
0,Agripada,2.0
1,Altamount Road,2.0
2,"Amboli, Mumbai",5.0
3,Amrut Nagar,2.0
4,Antop Hill,5.0
5,Anushakti Nagar,5.0
6,Asalfa,5.0
7,Badhwar Park,2.0
8,Baiganwadi,0.0
9,Ballard Estate,2.0


In [90]:
mumbai_cons.drop([92,93,94,95], inplace=True)

In [91]:
# create map
map_clusters = folium.Map(location=[19.0760, 72.8777], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_cons['latitude'], mumbai_cons['longitude'], mumbai_cons['neighborhood'], mumbai_cons['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
# CLUSTER 1
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 0]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
8,Baiganwadi,ATM,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Yoga Studio,Diner,Dhaba,Dessert Shop,0.0,Baiganwadi,19.061300,72.925800,https://en.wikipedia.org/wiki/Baiganwadi
38,Ferry Wharf,Boat or Ferry,Falafel Restaurant,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Zoo,Convenience Store,Donut Shop,Diner,Dhaba,0.0,Ferry Wharf,18.956382,72.849261,https://en.wikipedia.org/wiki/Ferry_Wharf
61,Manori,Farm,Zoo,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Yoga Studio,Diner,Dhaba,Dessert Shop,0.0,Manori,19.210000,72.790000,https://en.wikipedia.org/wiki/Manori
68,Nala Sopara,Park,Lake,Zoo,Electronics Store,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Food Court,Diner,Dhaba,0.0,Nala Sopara,19.416982,72.798733,https://en.wikipedia.org/wiki/Nala_Sopara


In [102]:
# CLUSTER 2 
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 1]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
55,Madh Island,Beach,Zoo,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Food Truck,Donut Shop,Diner,Dhaba,1.0,Madh Island,19.132283,72.794785,https://en.wikipedia.org/wiki/Madh_Island


In [103]:
# CLUSTER 3 - Kabir's current neighbourhood cluster
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 2]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
0,Agripada,Bakery,Indian Restaurant,Coffee Shop,Gym,Restaurant,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dhaba,2.0,Agripada,18.977700,72.827300,https://en.wikipedia.org/wiki/Agripada
1,Altamount Road,Café,Pizza Place,Bakery,Bookstore,Japanese Restaurant,Furniture / Home Store,Snack Place,Breakfast Spot,Brewery,Sandwich Place,Salon / Barbershop,Bus Station,Restaurant,Indian Restaurant,Italian Restaurant,2.0,Altamount Road,18.965200,72.808700,https://en.wikipedia.org/wiki/Altamount_Road
3,Amrut Nagar,Indian Restaurant,Café,Restaurant,Asian Restaurant,Shopping Mall,Electronics Store,Fast Food Restaurant,Bowling Alley,Brewery,Sandwich Place,Mediterranean Restaurant,Clothing Store,Bookstore,Falafel Restaurant,Chinese Restaurant,2.0,Amrut Nagar,19.102077,72.912835,https://en.wikipedia.org/wiki/Amrut_Nagar
7,Badhwar Park,Indian Restaurant,Café,Coffee Shop,Diner,Tennis Court,Restaurant,Hotel,German Restaurant,Falafel Restaurant,Middle Eastern Restaurant,Flea Market,Furniture / Home Store,Chaat Place,Chinese Restaurant,Stadium,2.0,Badhwar Park,18.921144,72.827325,https://en.wikipedia.org/wiki/Badhwar_Park
9,Ballard Estate,Convenience Store,Harbor / Marina,Hotel,Home Service,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Food,Dhaba,2.0,Ballard Estate,18.950000,72.840000,https://en.wikipedia.org/wiki/Ballard_Estate
10,Bangur Nagar,Smoke Shop,Café,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Zoo,Food,Diner,Dhaba,Dessert Shop,2.0,Bangur Nagar,19.167362,72.832252,https://en.wikipedia.org/wiki/Bangur_Nagar
13,Breach Candy,Café,Bakery,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Coffee Shop,Pizza Place,Electronics Store,Chinese Restaurant,Donut Shop,Frozen Yogurt Shop,Bus Station,Salon / Barbershop,Dessert Shop,Department Store,2.0,Breach Candy,18.967000,72.805000,https://en.wikipedia.org/wiki/Breach_Candy
14,Byculla,Indian Restaurant,Zoo,Pizza Place,Bakery,Furniture / Home Store,History Museum,Park,Pharmacy,Hotel,Track,Dhaba,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,2.0,Byculla,18.980000,72.835000,https://en.wikipedia.org/wiki/Byculla
15,C.G.S. colony,Pizza Place,Café,Ice Cream Shop,Garden,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Donut Shop,Food,Diner,Dhaba,2.0,C.G.S. colony,19.016378,72.856629,https://en.wikipedia.org/wiki/C.G.S._colony
17,"Chandanwadi, Mumbai",Indian Restaurant,Café,Bakery,Chinese Restaurant,Bar,Train Station,Coffee Shop,Multiplex,Italian Restaurant,Music Store,Cheese Shop,Fish Market,Fast Food Restaurant,Farmers Market,Farm,2.0,"Chandanwadi, Mumbai",18.946132,72.825736,"https://en.wikipedia.org/wiki/Chandanwadi,_Mumbai"


In [98]:
#
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 3]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
22,Colaba,Garden,Zoo,Electronics Store,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Food Court,Diner,Dhaba,Dessert Shop,3.0,Colaba,18.91,72.81,https://en.wikipedia.org/wiki/Colaba
24,Cuffe Parade,Garden,Zoo,Electronics Store,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop,Food Court,Diner,Dhaba,Dessert Shop,3.0,Cuffe Parade,18.91,72.81,https://en.wikipedia.org/wiki/Cuffe_Parade


In [104]:
# CLUSTER 5
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 4]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
70,Navy Nagar,Bakery,Bus Station,Zoo,Food Truck,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Donut Shop,Diner,4.0,Navy Nagar,18.9012,72.8101,https://en.wikipedia.org/wiki/Navy_Nagar
88,Virar,Bakery,Zoo,Falafel Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Electronics Store,Food Truck,Donut Shop,Diner,Dhaba,4.0,Virar,19.4700,72.8000,https://en.wikipedia.org/wiki/Virar


In [105]:
# CLUSTER 6
mumbai_cons.loc[mumbai_cons['Cluster Labels'] == 5]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,Cluster Labels,neighborhood,latitude,longitude,link
2,"Amboli, Mumbai",Chinese Restaurant,Indian Restaurant,Coffee Shop,Sandwich Place,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Food,5.0,"Amboli, Mumbai",19.129300,72.843400,"https://en.wikipedia.org/wiki/Amboli,_Mumbai"
4,Antop Hill,Indian Restaurant,Gym / Fitness Center,Trail,Grocery Store,Zoo,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Dessert Shop,Diner,5.0,Antop Hill,19.022220,72.866564,https://en.wikipedia.org/wiki/Antop_Hill
5,Anushakti Nagar,Indian Restaurant,Convenience Store,Lake,Playground,Creperie,Cupcake Shop,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Cosmetics Shop,5.0,Anushakti Nagar,19.036390,72.922500,https://en.wikipedia.org/wiki/Anushakti_Nagar
6,Asalfa,Campground,Motorcycle Shop,Indian Restaurant,Electronics Store,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Zoo,Food Court,Donut Shop,Diner,5.0,Asalfa,19.091000,72.901000,https://en.wikipedia.org/wiki/Asalfa
11,Bhuleshwar,Indian Restaurant,Food,American Restaurant,Ice Cream Shop,Market,Fast Food Restaurant,Restaurant,Electronics Store,Flower Shop,Flea Market,Fish Market,Farmers Market,Farm,Falafel Restaurant,Zoo,5.0,Bhuleshwar,18.950000,72.830000,https://en.wikipedia.org/wiki/Bhuleshwar
12,Bori Bunder,Indian Restaurant,Bar,Food Truck,Fast Food Restaurant,Seafood Restaurant,Train Station,Sandwich Place,Irani Cafe,Asian Restaurant,Chinese Restaurant,Café,Multiplex,Fish Market,Farmers Market,Farm,5.0,Bori Bunder,18.940000,72.835300,https://en.wikipedia.org/wiki/Bori_Bunder
16,Cavel,Indian Restaurant,Café,Cheese Shop,Music Store,Train Station,Coffee Shop,Multiplex,Chinese Restaurant,Bar,Bakery,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,5.0,Cavel,18.947400,72.827200,https://en.wikipedia.org/wiki/Cavel
19,Chira Bazaar,Indian Restaurant,Café,Train Station,Cheese Shop,Zoo,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Donut Shop,5.0,Chira Bazaar,18.948140,72.825462,https://en.wikipedia.org/wiki/Chira_Bazaar
20,Chor Bazaar,Indian Restaurant,Ice Cream Shop,Antique Shop,Dessert Shop,Arts & Crafts Store,Asian Restaurant,Juice Bar,Restaurant,Zoo,Falafel Restaurant,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,5.0,Chor Bazaar,18.960321,72.827176,https://en.wikipedia.org/wiki/Chor_Bazaar
26,Currey Road railway station,Indian Restaurant,Plaza,Coffee Shop,Donut Shop,Flower Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Diner,Food Court,Dhaba,5.0,Currey Road railway station,18.994000,72.833000,https://en.wikipedia.org/wiki/Currey_Road_rail...
